In [18]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing import image
import tensorflow.keras as keras


# Loading the dataset
We will first load in the csv dataset that labels the images with their corresponding labels. We will convert them to lists.

In [19]:
# read the csv
df = pd.read_csv('Training_set.csv')
files = df['filename'].tolist()
labels = df['label'].tolist()

# Getting the image paths
We will get the path of the images in the dataset.

In [20]:
image_path = './train/'

image_paths = [os.path.join(image_path, f) for f in files]

In [21]:
image_paths[:5]

['./train/Image_1.jpg',
 './train/Image_2.jpg',
 './train/Image_3.jpg',
 './train/Image_4.jpg',
 './train/Image_5.jpg']

# Iteration through the images.
We will now iterate and process the images. This is done by resizing them to (28, 28) and normalizing them, so that they can be fed into the model. Then we will add it to the X list.

In [50]:
X = []
for image_path in image_paths:
    img = image.load_img(image_path, target_size=(28, 28))
    img = image.img_to_array(img)
    img = img / 255.0
    X.append(img)
X = np.array(X)
X.shape

(6499, 28, 28, 3)

Now we will get the `y` labels. We will turn it into an np array.

In [51]:
y = np.array(labels)
y

array(['SOUTHERN DOGFACE', 'ADONIS', 'BROWN SIPROETA', ..., 'APPOLLO',
       'ELBOWED PIERROT', 'ATALA'], dtype='<U25')

# Encoding the labels
This is a function to convert the unique labels to numbers. This is so that we can use them in the model. Models don't understand strings.

In [52]:
def label_encoder(y):
  """
  Converts string labels in a list to corresponding numbers based on their order of appearance.

  Args:
    y: A list of string labels.

  Returns:
    A list of integers representing the corresponding numbers for each label.
    A dictionary mapping original string labels to assigned numbers.
  """

  label_to_number = {}
  counter = 0
  numbered_y = []
  for label in y:
    if label not in label_to_number:
      label_to_number[label] = counter
      counter += 1
    numbered_y.append(label_to_number[label])

  return numbered_y, label_to_number

In [53]:
y = label_encoder(y)

In [54]:
y = np.array(y[0])
y_to_number = y[1]

This will get the vocabulary of the labels. The size of it is 74. This will be important later.

In [55]:
max(y)

74

# Making the model
We will make the model. We will use the `Sequential` model. We will use `Conv2D`, `MaxPooling2D`, `Flatten`, `Dense` layers. We will use `adam` as the optimizer. We will use `Softmax` as the final layer for the model. This will make it a Convoluted Neural Network.

In [56]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(75, activation='softmax')
])

# Compiling the model
We will compile the model. We will use `sparse_categorical_crossentropy` as the loss function which is suited for multi-class classification or the use of `softmax`. We will use `accuracy` as the metric. We will use `adam` as the optimizer to help it lower the loss.

In [57]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
This will show how many parameters are in our model, in this case it is going to be 233,995 parameters or 914 kb worth of weights and bias.

In [58]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 1600)              0         
                                                                 
 dense_6 (Dense)             (None, 128)              

# Training
We will train our model on the first 100 images so it will be easier and faster to train. We will train for 12 epochs.

In [59]:
model.fit(X[200:], y[200:], epochs=10)

Epoch 1/10
197/197 [==============================] - 2s 10ms/step - loss: 3.8037 - accuracy: 0.0878
Epoch 2/10
197/197 [==============================] - 2s 11ms/step - loss: 2.6397 - accuracy: 0.2916
Epoch 3/10
197/197 [==============================] - 2s 11ms/step - loss: 2.1570 - accuracy: 0.4088
Epoch 4/10
197/197 [==============================] - 2s 11ms/step - loss: 1.8728 - accuracy: 0.4839
Epoch 5/10
197/197 [==============================] - 2s 11ms/step - loss: 1.6678 - accuracy: 0.5388
Epoch 6/10
197/197 [==============================] - 2s 11ms/step - loss: 1.4885 - accuracy: 0.5801
Epoch 7/10
197/197 [==============================] - 2s 11ms/step - loss: 1.3288 - accuracy: 0.6180
Epoch 8/10
197/197 [==============================] - 2s 11ms/step - loss: 1.1956 - accuracy: 0.6565
Epoch 9/10
197/197 [==============================] - 2s 11ms/step - loss: 1.0868 - accuracy: 0.6842
Epoch 10/10
197/197 [==============================] - 2s 11ms/step - loss: 0.9867 - accura

We will get the loss and accuracy of the model by evaluating it.

In [60]:
loss, accuracy = model.evaluate(X[2:], y[2:])
print(f'Loss: {loss}, Accuracy: {accuracy}')

204/204 [==============================] - 1s 4ms/step - loss: 0.8987 - accuracy: 0.7383
Loss: 0.8986737132072449, Accuracy: 0.7383407950401306


# Now we will make the prediction

In [61]:
new_image = image.load_img('./train/Image_10.jpg', target_size=(28, 28))
new_image = image.img_to_array(new_image)
new_image = new_image / 255.0


The giant output of numbers is each label with the corresponding probability of the image being that label. The highest probability is the predicted label. In this case it is label `7`.

In [62]:
prediction = model.predict(np.array([new_image]))
print(prediction)
print(f'Prediction: {np.argmax(prediction)}')
# print label corresponding to prediction
# Assuming you have the prediction stored in the variable 'prediction'
label = np.argmax(prediction)
print(f'Label corresponding to prediction: {label}')

1/1 [==============================] - 0s 46ms/step
[[2.43623643e-13 6.36725450e-10 3.98795555e-06 4.80462819e-01
  6.11516839e-07 3.72646718e-05 1.66915967e-10 9.13807773e-04
  8.23928981e-10 2.00935624e-09 2.70506528e-09 3.59644837e-05
  5.79805237e-05 3.68725602e-03 1.85397364e-10 2.83455842e-10
  7.23105855e-04 1.84829914e-05 2.88832166e-06 4.03661744e-07
  1.75000227e-03 1.25619599e-12 2.10871492e-02 2.88111082e-08
  6.28776888e-07 1.82032245e-09 2.23713315e-07 7.33753433e-04
  7.35761940e-09 5.73871148e-07 2.70124478e-09 2.66308058e-03
  8.72020493e-04 8.37439857e-03 1.74077968e-07 1.59148076e-05
  5.06016149e-05 7.16417417e-05 2.24805990e-05 2.37360075e-01
  9.84985745e-05 2.40755912e-06 6.98649600e-08 8.93153716e-04
  9.97106895e-07 2.05631223e-09 2.60375174e-07 2.54428723e-09
  2.67003173e-07 2.35449335e-07 1.55133804e-07 1.93688720e-06
  1.33492313e-02 4.32315096e-02 1.12112255e-11 9.14368081e-10
  1.56061396e-05 1.58536693e-07 1.94143271e-10 6.57165433e-10
  1.61873970e-07 1